<a href="https://colab.research.google.com/github/T-NIKHIL/Simple-Heat-Exchange-Model/blob/main/hw8_p5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

% Tushar Nichakawade
% tnichak1

% Acknowledgements to Nikhil Thota for help in reviewing this code and
% making sure it makes sense/works

clear all;
clc;

% Define the initial microstate
             %L R
microstate = [1 4; % E = 1
              1 0; % E = 2
              1 0; % E = 3 
              1 0];% E = 4 

% Run the heat simulation using timesimulation function below         
[Eleft,Eright]= timesimulation(100000,microstate);

% Generate plots for Barrick 9.1
figure()
subplot(3,1,1)
plot(1:1:100,Eleft(1:100)-Eright(1:100))
xlabel('Time Steps')
ylabel('E_L - E_R')
subplot(3,1,2)
plot(1:1:1000,Eleft(1:1000)-Eright(1:1000))
xlabel('Time Steps')
ylabel('E_L - E_R')
subplot(3,1,3)
plot(1:1:10000,Eleft(1:10000)-Eright(1:10000))
xlabel('Time Steps')
ylabel('E_L - E_R')

% Generate autocorrelation plot for 9.2
autoCdata = autocorrelation(Eleft,200);
figure()
plot(1:1:200,autoCdata)
xlabel('Time Separation (\tau)')
ylabel('Autocorrerelation E_L')

% Generate plots for Barrick 9.3
figure()
title('Histogram for 100000 timesteps')
subplot(3,1,1)
histogram(Eleft)
title('E_L')
xlabel('Total Energy')
ylabel('Frequency')
subplot(3,1,2)
histogram(Eright)
title('E_R')
xlabel('Total Energy')
ylabel('Frequency')
subplot(3,1,3)
histogram(Eleft-Eright)
title('E_L-E_R')
xlabel('Total Energy')
ylabel('Frequency')

% Generate microcannonical ensemble for the heat exchange model (10e5 steps)
% See heatensemble function below
ensemble = heatensemble(100000);

% Generate plots for Barrick 9.4
figure()
subplot(2,1,1)
histogram(Eleft)
title('E_L - Heat Exchange Simulation')
xlabel('Total Energy')
ylabel('Frequency')
subplot(2,1,2)
histogram(ensemble)
title('E_L - Microcanonical Ensemble')
xlabel('Total Energy')
ylabel('Frequency')

% Output some stuff
disp('Average for Heat Exchange Simulation')
disp(mean(Eleft))
disp('Average for Microcanonical Ensemble')
disp(mean(ensemble))
disp('StdDev for Heat Exchange Simulation')
disp(std(Eleft))
disp('StdDev for Microcanonical Ensemble')
disp(std(ensemble))

% Time dependent heat simulation (hard coded, not efficient, but works)
function [Eleft,Eright] = timesimulation(timesteps,microstate)

% Define book-keeping matrices 
leftstates = zeros(4,timesteps);
rightstates = zeros(4,timesteps);

% Define initial values of book-keeping matrices
leftstates(:,1) = microstate(:,1);
rightstates(:,1) = microstate(:,2);

for i=1:(timesteps-1)
     
    % Pick two particles using particle picker function (see below)
    particle_loc = particle_finder(microstate);
        
        % Constraints for particle movement
    
        % If E = 4 for one particle, decrease its energy & increase other
        if particle_loc(1,1) == 4 && particle_loc(2,1) < 4
           microstate(particle_loc(1,1),particle_loc(1,2)) = microstate(particle_loc(1,1),particle_loc(1,2)) - 1;
           microstate(particle_loc(1,1)-1,particle_loc(1,2)) = microstate(particle_loc(1,1)-1,particle_loc(1,2)) + 1;
           microstate(particle_loc(2,1),particle_loc(2,2)) = microstate(particle_loc(2,1),particle_loc(2,2)) - 1;
           microstate(particle_loc(2,1)+1,particle_loc(2,2)) = microstate(particle_loc(2,1)+1,particle_loc(2,2)) + 1;
           leftstates(:,i+1) = microstate(:,1);
           rightstates(:,i+1) = microstate(:,2);
       
        % If E = 4 for one particle, decrease its energy & increase other
        elseif particle_loc(1,1) < 4 && particle_loc(2,1) == 4
           microstate(particle_loc(1,1),particle_loc(1,2)) = microstate(particle_loc(1,1),particle_loc(1,2)) - 1;
           microstate(particle_loc(1,1)+1,particle_loc(1,2)) = microstate(particle_loc(1,1)+1,particle_loc(1,2)) + 1;
           microstate(particle_loc(2,1),particle_loc(2,2)) = microstate(particle_loc(2,1),particle_loc(2,2)) - 1;
           microstate(particle_loc(2,1)-1,particle_loc(2,2)) = microstate(particle_loc(2,1)-1,particle_loc(2,2)) + 1;
           leftstates(:,i+1) = microstate(:,1);
           rightstates(:,i+1) = microstate(:,2);
        
        % If E = 1 for one particle, increase its energy & decrease other
        elseif particle_loc(1,1) == 1 && particle_loc(2,1) > 1
           microstate(particle_loc(1,1),particle_loc(1,2)) = microstate(particle_loc(1,1),particle_loc(1,2)) - 1;
           microstate(particle_loc(1,1)+1,particle_loc(1,2)) = microstate(particle_loc(1,1)+1,particle_loc(1,2)) + 1;
           microstate(particle_loc(2,1),particle_loc(2,2)) = microstate(particle_loc(2,1),particle_loc(2,2)) - 1;
           microstate(particle_loc(2,1)-1,particle_loc(2,2)) = microstate(particle_loc(2,1)-1,particle_loc(2,2)) + 1;
           leftstates(:,i+1) = microstate(:,1);
           rightstates(:,i+1) = microstate(:,2);
        
        % If E = 1 for one particle, increase its energy & decrease other
        elseif particle_loc(2,1) == 1 && particle_loc(1,1) > 1
           microstate(particle_loc(1,1),particle_loc(1,2)) = microstate(particle_loc(1,1),particle_loc(1,2)) - 1;
           microstate(particle_loc(1,1)-1,particle_loc(1,2)) = microstate(particle_loc(1,1)-1,particle_loc(1,2)) + 1;
           microstate(particle_loc(2,1),particle_loc(2,2)) = microstate(particle_loc(2,1),particle_loc(2,2)) - 1;
           microstate(particle_loc(2,1)+1,particle_loc(2,2)) = microstate(particle_loc(2,1)+1,particle_loc(2,2)) + 1;
           leftstates(:,i+1) = microstate(:,1);
           rightstates(:,i+1) = microstate(:,2);
        
        % If both particles have E = 4, no energy change occurs
        elseif particle_loc(1,1) == 4 && particle_loc(2,1) == 4
            leftstates(:,i+1) = leftstates(:,i);
            rightstates(:,i+1) = rightstates(:,i);
            
        % If both particles have E = 1, no energy change occurs
        elseif particle_loc(1,1) == 1 && particle_loc(2,1) == 1
            leftstates(:,i+1) = leftstates(:,i);
            rightstates(:,i+1) = rightstates(:,i);
            
        % If the particles have intermediate energy values (2,3)
        else
           % Randomly asign an energy change direction (+1 or -1) 
           energychange = randsample([-1, 1],1,true);
           
           % Change the energy of particles according as randomly assigned
           microstate(particle_loc(1,1),particle_loc(1,2)) = microstate(particle_loc(1,1),particle_loc(1,2)) - 1;
           microstate(particle_loc(1,1)+energychange,particle_loc(1,2)) = microstate(particle_loc(1,1)+energychange,particle_loc(1,2)) + 1;
           microstate(particle_loc(2,1),particle_loc(2,2)) = microstate(particle_loc(2,1),particle_loc(2,2)) - 1;
           microstate(particle_loc(2,1)-energychange,particle_loc(2,2)) = microstate(particle_loc(2,1)-energychange,particle_loc(2,2)) + 1;
           leftstates(:,i+1) = microstate(:,1);
           rightstates(:,i+1) = microstate(:,2);
        end       
end
E = zeros(4,timesteps);
E(1,:) = 1;
E(2,:) = 2;
E(3,:) = 3;
E(4,:) = 4;

Eleft = dot(leftstates,E);
Eright = dot(rightstates,E);
end

% Particle picking program
function particle_loc = particle_finder(state)
    for i=1:10000
    % Randomly pick particle side (L = 1, R = 2)
     pick_side = randi([1 2], 1, 2);
    
    % Randomly pick two particle indices (1,2,3,4)
     pick_particle = randi([1 4], 1, 2);
     
        % Check that the chosen particle exists (has value > 0 in state matrix)
        if state(pick_particle(1,1),pick_side(1,1)) > 0 && state(pick_particle(1,2), pick_side(1,2)) > 0
        
            % If we picked the same energy lvl & side, ensure > 2 particles
            if pick_particle(1,1) == pick_particle(1,2) && pick_side(1,1) == pick_side(1,2)
                if state(pick_particle(1,1),pick_side(1,1)) > 2 
                    particle_loc = [pick_particle(1,1),pick_side(1,1);
                                    pick_particle(1,2), pick_side(1,2)];
                break;
                end
            % If not same energy level & side, pick the particles
            else 
                particle_loc = [pick_particle(1,1),pick_side(1,1);
                                pick_particle(1,2), pick_side(1,2)];
            break;
            end  
        end 
    end 
end 

% Function for autocorrerelation function, via equation 9.1 Barrick
function [autoCdata] = autocorrelation(Ediff,tau)
   % Calculate the average energy <Y>
   average = sum(Ediff)/size(Ediff,2);
   
   % Define solution matrices
   numerator = zeros(1,tau);
   denominator = zeros(1,tau);
   autoCdata = zeros(1,tau);
   
   % Iterate through values and calculate the correrelation at each tau
   for i = 1:tau
       
       % Calculate each numerator and denominator value
       for n = 0:(size(Ediff,2)-i)
       numerator(1,n+1) = (Ediff(1,n+1)-average)*(Ediff(1,n+i));
       denominator(1,n+1) = (Ediff(1,n+1)-average)^2;
       end
       
       % Calculate the correrelation value
       autoCdata(1,i) = sum(numerator)/sum(denominator);
   end
end

% This function generates our microcannonical ensemble based on random
% numbers and the probability of each state
function ensemble = heatensemble(timesteps)
    % Define solution matrix
    ensemble = zeros(1,timesteps);
    
    % Generate random number from 1-1428
    data = randi([1 1428],1,timesteps);
    
    % Depending on the value, assign an energy level to our solution matrix
    for i = 1:timesteps
        if data(1,i) <= 44
            ensemble(1,i) = 4;
        elseif data(1,i) <= 204
            ensemble(1,i) = 5;
        elseif data(1,i) <= 514
            ensemble(1,i) = 6;
        elseif data(1,i) <= 914
            ensemble(1,i) = 7;
        elseif data(1,i) <= 1224
            ensemble(1,i) = 8;
        elseif data(1,i) <= 1384
            ensemble(1,i) = 9;
        elseif data(1,i) <= 1428
            ensemble(1,i) = 10;
        end 
    end 
end


 <<< USER INPUT SECTION >>>

In [6]:
# Enter the initial microstate
# Example : [1,4] E=1
#           [1,0] E=2
#           [1,0] E=3
#           [1,0] E=4
microstate = np.array([[1,4],[1,0],[1,0],[1,0]])

# Input number of timesteps you want the simulation to run for
n = 100000

First lets import the classes we would be using

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random 

Next lets define some common arrays that we will use throughout the program

In [ ]:
# We will use this when we have to select a side to perform a random movement 
threeSidedDice = np.array([0,1,2])
# We will use this when we want to select an energy level in the system
energies = np.array([1,2,3,4])
# For appending the microstate to the growing ensemble
ensemble = microstate

Class function ?? Do we need  ?? 

In [ ]:
# Creating a class for microstate
class microstate :
  def __init__(self,)

  def calculate_energy_left():

  def calculate_energy_right():


This block contains the list of functions used in the program

In [4]:
def pickParticle(microstate):
  return 

# If output is 0 (then left side)
# If output is 1 (then right side)
# If ouptupt is 2 (then one from each side)
def chooseSide():
  return random.choice(threeSidedDice)

def chooseEnergyLevel():
  energy1 = random.choice(energies)
  energy2 = random.choice(energies)
  return energy1, energy2

def checkValidEnergySelection(microState,energy1,energy2,whichSide):

  # Get the number of particles in the energy levels based on the side
  if whichSide == 0
    numParticlesEnergy1 = microstate[energy1 - 1,0]
    numParticlesEnergy2 = microstate[energy2 - 1,0]
  elif whichSide == 1:
    numParticlesEnergy1 = microstate[energy1 - 1,1]
    numParticlesEnergy2 = microstate[energy2 - 1,1]
  else:
    numParticlesEnergy1 = microstate[energy1 - 1,0]
    numParticlesEnergy2 = microstate[energy2 - 1,1]

  # if numParticlesEnergy1 and numParticlesEnergy2 both greater than 0 then we can proceed with further evaluation
  if (numParticlesEnergy1 > 0) and (numParticlesEnergy2 > 0):
    # If we are working in the same side and we picked the same energy level then we have to make sure sure we have greater than 2 particles
    if (whichSide == 0 or whichSide == 1) and (energy1 == energy2):
      # Just choose any variable numParticlesEnergy1 or numParticlesEnergy2 as they both reference the same energy level on same side
      if (numParticlesEnergy1 > 2):
        return 1
      else:
        return 0      
    # If you are working in different sides then it is a valid energy selection
    else:
      return 1  
  else :
    # Return 0 if invalid move
    return 0

# If energy selction is valid proceed to perfom move
def performMove(microstate,enesemble,whichSide,energy1,energy2):
  # Lets first deal with extreme energy levels 
  

  return microstate, enesemble
